In [2]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
from preprocess_days_stats import preprocess_cumulative_stats, preprocess_match_days
import warnings
# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")


df_giornate = preprocess_match_days(r"c:\Users\Hp\Documents\Serie_A_dump\csv_serie_a")

Reading file: I1 (0).csv
Reading file: I1 (1).csv
Reading file: I1 (10).csv
Reading file: I1 (11).csv
Reading file: I1 (12).csv
Reading file: I1 (13).csv
Reading file: I1 (14).csv
Reading file: I1 (15).csv
Reading file: I1 (16).csv
Reading file: I1 (17).csv
Reading file: I1 (18).csv
Reading file: I1 (2).csv
Reading file: I1 (3).csv
Reading file: I1 (4).csv
Reading file: I1 (5).csv
Reading file: I1 (6).csv
Reading file: I1 (7).csv
Reading file: I1 (8).csv
Reading file: I1 (9).csv
preprocessing finished!


In [42]:
import csv

# Nome del file CSV
nome_file = "results/baseline_KNN.csv"

# Apri il file CSV in modalità scrittura
with open(nome_file, mode='w', newline='') as file_csv:
    # Creazione dell'oggetto writer
    writer = csv.writer(file_csv)

    # Scrivi l'intestazione nel file CSV
    writer.writerow(['giorni_cumulativi', 'vanumero_colonne', 'vicini', 'val_accuracy', 'train_accuracy'])

# KNN baseline

In [43]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score
from preprocess_features import preprocess_columns

for giorni_cumulativi in range(10):
    giorni_cumulativi = giorni_cumulativi+1
    df_Serie_A_days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)
    for numero_colonne in ['all','less','few']:    
        X_train_norm, X_valid_norm, X_test_norm, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded = preprocess_columns(
            df_Serie_A_days,numero_colonne,giorni_cumulativi)

        best_accuracy=0
        for vicini in range(3,40): #prendiamo diversi numeri di vicini possibili
            knn_model = KNeighborsClassifier(n_neighbors=3*vicini) #addestriamo il modello con questo numero di vicini 
            knn_model.fit(X_train_norm, Train_labels_encoded) 
            y_valid = knn_model.predict(X_valid_norm)
            val_accuracy = round(accuracy_score(Valid_labels_encoded, y_valid),2)
            y_train = knn_model.predict(X_train_norm)
            train_accuracy = round(accuracy_score(Train_labels_encoded, y_train),2)

            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                
                # Scrivi i dati nel file CSV
                with open(nome_file, mode='a', newline='') as file_csv:
                    writer = csv.writer(file_csv)
                    writer.writerow([giorni_cumulativi, numero_colonne, vicini*3, val_accuracy, train_accuracy])            

preprocess finished
utilizzando tutte le features
utilizzando meno features
utilizzando poche features
preprocess finished
utilizzando tutte le features
utilizzando meno features
utilizzando poche features
preprocess finished
utilizzando tutte le features
utilizzando meno features
utilizzando poche features
preprocess finished
utilizzando tutte le features
utilizzando meno features
utilizzando poche features
preprocess finished
utilizzando tutte le features
utilizzando meno features
utilizzando poche features
preprocess finished
utilizzando tutte le features
utilizzando meno features
utilizzando poche features
preprocess finished
utilizzando tutte le features
utilizzando meno features
utilizzando poche features
preprocess finished
utilizzando tutte le features
utilizzando meno features
utilizzando poche features
preprocess finished
utilizzando tutte le features
utilizzando meno features
utilizzando poche features
preprocess finished
utilizzando tutte le features
utilizzando meno featur

In [45]:
baseline_KNN_df = pd.read_csv(r'baseline_KNN.csv').sort_values(['val_accuracy'],ascending=False)
baseline_KNN_df.head(10)

,giorni_cumulativi,vanumero_colonne,vicini,val_accuracy,train_accuracy
14,1,few,72,0.53,0.51
13,1,few,63,0.52,0.52
89,6,all,117,0.52,0.50
63,4,less,117,0.51,0.51
34,3,all,45,0.51,0.51
149,9,less,105,0.51,0.50
43,3,less,57,0.51,0.51
124,7,few,69,0.51,0.51
49,3,few,63,0.51,0.51
97,6,less,63,0.51,0.51


In [46]:
baseline_KNN_df.sort_values(['train_accuracy'],ascending=False).head(10)

,giorni_cumulativi,vanumero_colonne,vicini,val_accuracy,train_accuracy
118,7,few,9,0.39,0.58
0,1,all,9,0.46,0.58
150,9,few,9,0.44,0.58
44,3,few,9,0.44,0.58
10,1,few,9,0.48,0.58
24,2,few,9,0.45,0.57
35,3,less,9,0.41,0.57
105,7,all,9,0.39,0.57
19,2,less,9,0.44,0.57
15,2,all,9,0.46,0.57


# Dense neural network

In [21]:
import tensorflow as tf 

# Create a function to implement a ModelCheckpoint callback with a specific filename 
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                            verbose=0, # only output a limited amount of text
                                            monitor='val_loss',
                                            save_best_only=True) # save only the best model to file

## Iniziamo con poche features e 3 giorni cumulativi

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score
from preprocess_features import preprocess_columns

giorni_cumulativi=3
numero_colonne='few'
df_Serie_A_days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)

X_train_norm, X_valid_norm, X_test_norm, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded = preprocess_columns(
            df_Serie_A_days,numero_colonne,giorni_cumulativi)

preprocess finished
utilizzando poche features


In [24]:
len(X_train_norm), len(X_valid_norm), len(X_test_norm)

(5919, 345, 696)

In [23]:
import tensorflow as tf 
from tensorflow.keras import layers
from helper_functions_tensorflow import CSVLoggerCallback
import random 

#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

# creo il modello 
inputs = layers.Input(shape=(118,))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dropout(0.1)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.1)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(3, activation='softmax')(x)
model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

# compilo il modello 
model_1.compile(
    loss='sparse_categorical_crossentropy',
    metrics = 'accuracy',
    optimizer = tf.keras.optimizers.Adam()
)

# fitto il modello 
model_1.fit(
    x = X_train_norm,
    y = Train_labels_encoded,
    epochs = 12,
    validation_data = (X_valid_norm, Valid_labels_encoded),
    callbacks = [
        CSVLoggerCallback('results.csv','model_1_dense',overwrite=True), 
        #create_model_checkpoint(model_name= 'Dense_model_best')
                 ] 
)

Epoch 1/12
185/185 [==============================] - 2s 3ms/step - loss: 1.0482 - accuracy: 0.4602 - val_loss: 1.0157 - val_accuracy: 0.5043
Epoch 2/12
185/185 [==============================] - 0s 2ms/step - loss: 1.0117 - accuracy: 0.5001 - val_loss: 0.9945 - val_accuracy: 0.5159
Epoch 3/12
185/185 [==============================] - 0s 2ms/step - loss: 0.9864 - accuracy: 0.5278 - val_loss: 0.9883 - val_accuracy: 0.5420
Epoch 4/12
185/185 [==============================] - 0s 2ms/step - loss: 0.9704 - accuracy: 0.5359 - val_loss: 0.9904 - val_accuracy: 0.5304
Epoch 5/12
185/185 [==============================] - 0s 2ms/step - loss: 0.9599 - accuracy: 0.5400 - val_loss: 0.9871 - val_accuracy: 0.5333
Epoch 6/12
185/185 [==============================] - 0s 2ms/step - loss: 0.9515 - accuracy: 0.5464 - val_loss: 0.9983 - val_accuracy: 0.5188
Epoch 7/12
185/185 [==============================] - 0s 2ms/step - loss: 0.9394 - accuracy: 0.5567 - val_loss: 0.9963 - val_accuracy: 0.5188
Epoch 

In [20]:
model_1 = loaded_model = tf.keras.models.load_model("model_experiments\Dense_model_best")
model_1.evaluate(X_test_norm, Test_labels_encoded)

22/22 [==============================] - 0s 1ms/step - loss: 1.0116 - accuracy: 0.5115


[1.0115653276443481, 0.5114942789077759]